# Building a CNN Model to test Polarity

### Import Packages and Libraries

In [43]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import re
import gensim
from gensim import corpora
from gensim import similarities
from gensim import models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

### Dataset


In [103]:
data_file = "SHOPEE_MAYBELLINE_CLEAN_V2.csv"
data = pd.read_csv(data_file)
data.columns = data.columns.str.strip().str.replace(" ","_") # reason for doing this is cause cannot get columns with space into a list
# data.info()
# data.head()

# data.drop(columns=['Brand','Category','Product_Name','Price','Reviewer','Product_Purchase','Ratings','Date_Of_Review','Response', 'Topic'])
# review_list = data['Review'].tolist()
# polarity_list = data['Polarity'].tolist()

reviews = data['Review']
# polarity = data['Polarity']
# print (reviews)

review_docs = []
for each_reviews in reviews:
    temp = each_reviews.split(" ")
    review_docs.append(temp)
print (review_docs)

# Make sure all words are in lowercase
reviews_lower = [[each_word.lower() for each_word in each_review] for each_review in review_docs]
# print (reviews_lower)

# Use regular expressions to keep only allphabetical words
reviews_alpha = [[each_word for each_word in each_review if re.search('^[a-z]+$', each_word)] for each_review in reviews_lower]
# print (reviews_alpha)

# Remove stop words
stop_list = stopwords.words('english')
reviews_stop = [[each_word for each_word in each_review if each_word not in stop_list] for each_review in reviews_alpha]
# print (reviews_stop)

# Porter Stemming
stemmer = PorterStemmer()
reviews_stem = [[stemmer.stem(each_word) for each_word in each_review] for each_review in reviews_stop]
# print (reviews_stem)

all_data_cleaned = []
for each_sentence in reviews_stem:
    sentence = ""
    for each_word in each_sentence:
        sentence += each_word + " "
    sentence = sentence[0:-1]
    all_data_cleaned.append(sentence)
# print (all_data_cleaned)

reviews = all_data_cleaned
polarity = data['Polarity']

X_train, X_test, y_train, y_test = train_test_split(reviews, polarity, test_size=0.2, random_state=42)
     


[['fast', 'delivery', 'and', 'bubble', 'wrapped'], ['i', 'have', 'yet', 'to', 'try', 'it', 'out...............'], ['i', 'chooose', 'the', 'colour', 'very', 'dull', 'i', 'dont', 'like', 'to', 'use', 'it', 'anymore', '.'], ['but', 'the', 'i', 'get', 'on', 'flash', 'deal', 'so', 'still', 'worth', 'it', 'the', 'price'], ['shipping', 'took', 'one', 'week'], ['pretty', 'but', 'will', 'have', 'to', 'mix', 'color', 'to', 'wear', 'on', 'lips', 'heh', 'thanks', 'seller!'], ['packing', 'leak'], ['handy', 'for', 'travel'], ['good', 'deal', 'considering', 'the', 'brand'], ['package', 'arrived', 'sooner', 'than', 'expected', 'and', 'well', 'packaged', 'as', 'well'], ["can't", 'wait', 'to', 'test', 'this', 'out!'], ['overall', 'is', 'a', 'good', 'buy', 'to', 'try', 'the', 'product', '^^'], ['delivery', 'was', 'really', 'fast'], ['packing', 'for', 'small', 'item', 'fuss', 'free', 'and', 'neat'], ['however', 'product', 'is', 'really', 'sticky'], ['i', 'have', 'tried', 'other', 'colours', 'before', 'but

### Building the Model

In [100]:
dictionary = corpora.Dictionary(reviews_stem)
# print (dictionary)
max_sequence_length = (len(dictionary))

model = tf.keras.Sequential()
# model.add(layers.Conv2D(4, kernel_size = (5,5), 
#                         activation='relu', 
#                         input_shape=(max_sequence_length,)))
model.add(layers.Dense(10, input_shape = (max_sequence_length, ), activation='softmax'))
model.add(layers.Dense(1, activation = 'softmax'))
model.summary()


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 10)                3680      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 11        
Total params: 3,691
Trainable params: 3,691
Non-trainable params: 0
_________________________________________________________________


In [101]:
model.compile(optimizer=tf.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [102]:
epochs = 1
batch_size = 16
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'str'>"}), <class 'pandas.core.series.Series'>